In [1]:
%pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp312-cp312-win_amd64.whl (6.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp312-cp312-win_amd64.whl (4.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import random, json, numpy as np, torch, pymongo
import torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

SEED = 42
BATCH = 512
EPOCHS = 15
LR = 3e-4

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

import torch, subprocess, sys, os

print("torch.cuda.is_available():", torch.cuda.is_available())
print("torch.version.cuda:", torch.version.cuda)
print("GPU count:", torch.cuda.device_count())


torch.cuda.is_available(): True
torch.version.cuda: 12.1
GPU count: 1


In [8]:
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['dota']
matches_info = db['matches_info']

matches_raw = list(
    matches_info.find(
        {
            'players.9': {'$exists': True},
            'picks_bans': {'$exists': True}
        },
        {
            '_id': 0,
            'match_id': 1,
            'picks_bans': 1
        }
    )
)
print("Loaded", len(matches_raw), "matches with picks_bans")

max_id = max(
    h["hero_id"]
    for m in matches_raw
    for h in m["picks_bans"]
    if h.get("is_pick")
)
HERO_COUNT = max_id + 1
print("HERO_COUNT =", HERO_COUNT)
print('Loaded', len(matches_raw), 'matches')

Loaded 43439 matches with picks_bans
HERO_COUNT = 146
Loaded 43439 matches


In [9]:
from typing import List, Dict

def generate_examples_parallel(pb: List[Dict]):
    picks = [p for p in pb if p.get('is_pick')]
    rad = sorted([p for p in picks if p['team']==0], key=lambda x: x['order'])
    dire = sorted([p for p in picks if p['team']==1], key=lambda x: x['order'])
    phases = [(2,2),(2,2),(1,1)]
    ex, vis, r_i, d_i = [], [], 0, 0
    for r_n, d_n in phases:
        r_now, d_now = rad[r_i:r_i+r_n], dire[d_i:d_i+d_n]
        for i,p in enumerate(r_now):
            ex.append({'input_H': vis + [q['hero_id'] for q in r_now[:i]],
                       'team':0,'y':p['hero_id']})
        for i,p in enumerate(d_now):
            ex.append({'input_H': vis + [q['hero_id'] for q in d_now[:i]],
                       'team':1,'y':p['hero_id']})
        vis += [p['hero_id'] for p in r_now+d_now]
        r_i += r_n; d_i += d_n
    return ex

In [10]:
class DraftDataset(Dataset):
    def __init__(self, matches):
        if hasattr(matches, "to_dict"):
            matches = matches.to_dict("records")
        self.samples = []
        for m in matches:
            if "picks_bans" in m:
                self.samples += generate_examples_parallel(m["picks_bans"])
        random.shuffle(self.samples)

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]
        x = np.zeros(2*HERO_COUNT, dtype=np.float32)
        for h in s["input_H"]:
            x[h] = 1.0
        mask = np.zeros(HERO_COUNT, dtype=np.float32)
        mask[s["input_H"]] = 1.0
        return torch.tensor(x), torch.tensor(mask), torch.tensor(s["y"])


In [11]:
class DraftNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2 * HERO_COUNT, 512), torch.nn.SiLU(),
            torch.nn.Linear(512, 256),            torch.nn.SiLU(),
            torch.nn.Linear(256, HERO_COUNT)
        )

    def forward(self, x, mask):
        logits = self.net(x)
        return logits.masked_fill(mask.bool(), -1e9)

In [6]:
def train_model(matches):
    ds = DraftDataset(matches)
    train_size = int(0.9*len(ds))
    tr_ds, val_ds = random_split(ds, [train_size, len(ds)-train_size],
                                 generator=torch.Generator().manual_seed(SEED))
    dl_tr = DataLoader(tr_ds, batch_size=BATCH, shuffle=True, pin_memory=True)
    dl_val = DataLoader(val_ds, batch_size=BATCH, pin_memory=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = DraftNet().to(device)
    opt = torch.optim.AdamW(net.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        net.train()
        for x,mask,y in dl_tr:
            x,mask,y = x.to(device), mask.to(device), y.to(device)
            opt.zero_grad(); loss = loss_fn(net(x,mask), y); loss.backward(); opt.step()

        net.eval(); top1=top3=total=0
        with torch.no_grad():
            for x,mask,y in dl_val:
                x,mask,y = x.to(device), mask.to(device), y.to(device)
                logits = net(x,mask)
                _, idx = logits.topk(3,1)
                total += y.size(0)
                top1 += (idx[:,0]==y).sum().item()
                top3 += ((idx==y.unsqueeze(1)).any(1)).sum().item()
        print(f'E{epoch:02d} top1={top1/total:.3f} top3={top3/total:.3f}')

    torch.save(net.state_dict(), 'draftnet.pt')
    return net


In [7]:
net = train_model(matches_raw)
torch.save(net.state_dict(), "draftnet.pt")

E00 top1=0.065 top3=0.137
E01 top1=0.069 top3=0.145
E02 top1=0.069 top3=0.147
E03 top1=0.070 top3=0.147
E04 top1=0.069 top3=0.147
E05 top1=0.070 top3=0.146
E06 top1=0.070 top3=0.147
E07 top1=0.070 top3=0.147
E08 top1=0.070 top3=0.148
E09 top1=0.070 top3=0.147
E10 top1=0.070 top3=0.147
E11 top1=0.071 top3=0.147
E12 top1=0.070 top3=0.146
E13 top1=0.070 top3=0.146
E14 top1=0.070 top3=0.146


In [47]:
heroes_raw = list(db.heroes.find({}, {'_id': 0, 'id': 1, 'localized_name': 1}))
name2id = {h['localized_name']: h['id'] for h in heroes_raw}
id2name = {h['id']: h['localized_name'] for h in heroes_raw}

radiant_picks = [
    name2id['Lina'],
    name2id['Sniper'],
    name2id['Doom'],
    name2id['Oracle'],
]
dire_picks = [
    name2id['Morphling'],
    name2id['Zeus'],
    name2id['Axe'],
    name2id['Abaddon'],
]
dire_picks

[10, 22, 2, 102]

In [18]:
import torch, numpy as np, torch.nn.functional as F
HERO_COUNT = torch.load('draftnet.pt', weights_only=True)['net.0.weight'].shape[1] // 2

class DraftNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2*HERO_COUNT, 512), torch.nn.SiLU(),
            torch.nn.Linear(512, 256),          torch.nn.SiLU(),
            torch.nn.Linear(256, HERO_COUNT)
        )
    def forward(self, x, mask):
        logits = self.net(x)
        return logits.masked_fill(mask.bool(), -1e9)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = DraftNet().to(device)
model.load_state_dict(torch.load('draftnet.pt', map_location=device))
model.eval()

def recommend(my_picks, opp_picks, topk=5, T=1.0):
    x = np.zeros(2*HERO_COUNT, dtype=np.float32)
    for h in my_picks:  x[h] = 1.0
    for h in opp_picks: x[h+HERO_COUNT] = 1.0
    mask = np.zeros(HERO_COUNT, dtype=np.float32)
    mask[my_picks + opp_picks] = 1.0

    with torch.no_grad():
        logits = model(torch.tensor(x).unsqueeze(0).to(device),
                       torch.tensor(mask).unsqueeze(0).to(device)) / T
        probs = torch.softmax(logits, dim=1)[0].cpu().numpy()
        top = probs.argsort()[-topk:][::-1]
        return [(id2name[h], probs[h]) for h in top]


C:\Users\taras\AppData\Local\Temp\ipykernel_31140\1965879010.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('draftnet.pt', map_locatio

In [49]:
top = recommend(my_picks=dire_picks, opp_picks=radiant_picks, topk=5)
#top = recommend(my_picks=radiant_picks, opp_picks=dire_picks, topk=5)

for name, prob in top:
    print(f"{name:<15} {prob:.3f}")


Jakiro          0.035
Lion            0.029
Ancient Apparition 0.026
Queen of Pain   0.025
Rubick          0.025
